In [1]:
import re
import tools
import sqlite3
import zipfile

import numpy as np
import pandas as pd

from datetime import datetime

## Conhecer a base de dados dos acórdãos do TCU

A base de dados dos acórdãos do TCU foi disponibilizada para download público na plataforma Kaggle. Os dados dizem respeito aos acórdãos proferidos pelo TCU entre os anos de 1992 até 30/08/2019.
Acesse o endereço <https://www.kaggle.com/ferraz/acordaos-tcu> e baixe o arquivo no link Download (4GB) e salve na pasta '/dados'.
github: <https://github.com/netoferraz/acordaos-tcu>

In [ ]:
# Remover os comentário das linhas abaixo caso ainda não tenha feito a descompactação
# with zipfile.ZipFile('dados/330881_660826_bundle_archive.zip', 'r') as zip_ref:
#     zip_ref.extractall()

In [3]:
# Cria conexão com a base de datos sqlite
conn = sqlite3.connect('dados/tcu-acordaos.db')

In [4]:
# Cria o dataframe df_acordaos
start_time = datetime.now()
df_acordaos = pd.read_sql_query('SELECT * from acordaos', conn)
print('Duration: {}'.format(datetime.now() - start_time))

Duration: 0:00:13.333707


In [5]:
print(df_acordaos.shape, '\n')

# Mostra as colunas da base
print(df_acordaos.columns, '\n')

print(df_acordaos.info(), '\n')

(298942, 20) 

Index(['id', 'urn', 'ano_acordao', 'numero_acordao', 'relator', 'processo',
       'tipo_processo', 'data_sessao', 'numero_ata',
       'interessado_reponsavel_recorrente', 'entidade', 'representante_mp',
       'unidade_tecnica', 'repr_legal', 'assunto', 'sumario', 'acordao',
       'quorum', 'relatorio', 'voto'],
      dtype='object') 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298942 entries, 0 to 298941
Data columns (total 20 columns):
 #   Column                             Non-Null Count   Dtype 
---  ------                             --------------   ----- 
 0   id                                 298942 non-null  int64 
 1   urn                                298942 non-null  object
 2   ano_acordao                        298942 non-null  int64 
 3   numero_acordao                     298123 non-null  object
 4   relator                            298470 non-null  object
 5   processo                           298562 non-null  object
 6   tipo_processo   

In [6]:
# Transfora em string as colunas ano_acordao
df_acordaos['ano_acordao'] = df_acordaos['ano_acordao'].map(str)

df_acordaos.rename(str.upper, axis='columns', inplace=True)
df_acordaos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298942 entries, 0 to 298941
Data columns (total 20 columns):
 #   Column                             Non-Null Count   Dtype 
---  ------                             --------------   ----- 
 0   ID                                 298942 non-null  int64 
 1   URN                                298942 non-null  object
 2   ANO_ACORDAO                        298942 non-null  object
 3   NUMERO_ACORDAO                     298123 non-null  object
 4   RELATOR                            298470 non-null  object
 5   PROCESSO                           298562 non-null  object
 6   TIPO_PROCESSO                      298470 non-null  object
 7   DATA_SESSAO                        298562 non-null  object
 8   NUMERO_ATA                         298470 non-null  object
 9   INTERESSADO_REPONSAVEL_RECORRENTE  297679 non-null  object
 10  ENTIDADE                           297777 non-null  object
 11  REPRESENTANTE_MP                   297854 non-null  

In [7]:
# Verifica a quantidade de registros marcados como None
print(df_acordaos[df_acordaos['ACORDAO'].values == None].shape[0])
print(df_acordaos[df_acordaos['RELATORIO'].values == None].shape[0])
print(df_acordaos[df_acordaos['VOTO'].values == None].shape[0])

# Converte os valores None para np.NaN
df_acordaos.loc[df_acordaos['ACORDAO'].values == None, 'ACORDAO'] = np.NaN
df_acordaos.loc[df_acordaos['RELATORIO'].values == None, 'RELATORIO'] = np.NaN
df_acordaos.loc[df_acordaos['VOTO'].values == None, 'VOTO'] = np.NaN

# Confirmação
print(df_acordaos[df_acordaos['ACORDAO'].values == None].shape[0])
print(df_acordaos[df_acordaos['RELATORIO'].values == None].shape[0])
print(df_acordaos[df_acordaos['VOTO'].values == None].shape[0])

495
29743
29743
0
0
0


In [8]:
# Retira colunas que não interessam para o trabalho
df_acordaos = df_acordaos.drop(['REPRESENTANTE_MP', 'UNIDADE_TECNICA', 'REPR_LEGAL', 'QUORUM', 
                                'INTERESSADO_REPONSAVEL_RECORRENTE'], axis=1)

In [9]:
def imprime_proc_sigilo():
    print('Qtde de acordãos sem número de processo: ', df_acordaos[df_acordaos.PROCESSO.isnull()].shape[0])
    print('Qtde de acordãos com tipo de processo SIGILOSO: ', df_acordaos[(df_acordaos.TIPO_PROCESSO == 'SIGILOSO')].shape[0])
    print('Qtde de acordãos com acordao SIGILOSO: ', df_acordaos[(df_acordaos.ACORDAO == 'SIGILOSO') | (df_acordaos.ACORDAO == 'None')].shape[0])
    print('Qtde de acordãos com voto SIGILOSO: ', df_acordaos[(df_acordaos.VOTO == 'SIGILOSO')  | (df_acordaos.VOTO == 'None')].shape[0])
    print('Qtde de acordãos com entidade SIGILOSO: ', df_acordaos[(df_acordaos.ENTIDADE == 'SIGILOSO')  | (df_acordaos.ENTIDADE == 'None')].shape[0])
    print('Qtde de acordãos com Número de Acórdão não utilizado: ', df_acordaos[df_acordaos.SUMARIO == 'Número de Acórdão não utilizado'].shape[0], '\n')
    
    qtdeSigiloso = df_acordaos[(df_acordaos['ENTIDADE'] == 'SIGILOSO') | (df_acordaos['TIPO_PROCESSO'].str.upper == 'SIGILOGO')
                               | (df_acordaos['SUMARIO'].str.upper == 'SIGILOGO')
                               | (df_acordaos['ACORDAO'].str.upper == 'SIGILOGO')
                               | (df_acordaos['RELATORIO'].str.upper == 'SIGILOGO')
                               | (df_acordaos['ENTIDADE'].str.upper == 'SIGILOGO')
                               | (df_acordaos['VOTO'].str.upper == 'SIGILOGO')
                               | (df_acordaos.SUMARIO == 'Número de Acórdão não utilizado')].shape[0]

    print('Quantidade de acordãos marcados com sigilo: ', qtdeSigiloso, '\n')
    del qtdeSigiloso

In [10]:
imprime_proc_sigilo()


print('Removendo os registros...')
df_acordaos = df_acordaos[df_acordaos.PROCESSO.notnull()]
print(df_acordaos.shape[0])
df_acordaos = df_acordaos.drop(df_acordaos.index[df_acordaos['TIPO_PROCESSO'] =='SIGILOSO'])
print(df_acordaos.shape[0])
df_acordaos = df_acordaos.drop(df_acordaos.index[(df_acordaos['ACORDAO'] == 'SIGILOSO') | (df_acordaos['ACORDAO'] == 'None')])
print(df_acordaos.shape[0])
df_acordaos = df_acordaos.drop(df_acordaos.index[(df_acordaos['VOTO'] == 'SIGILOSO') | (df_acordaos['VOTO'] == 'None')])
print(df_acordaos.shape[0])
df_acordaos = df_acordaos.drop(df_acordaos.index[(df_acordaos['ENTIDADE'] == 'SIGILOSO') | (df_acordaos['ENTIDADE'] == 'None')])
print(df_acordaos.shape[0])
df_acordaos = df_acordaos.drop(df_acordaos.index[(df_acordaos['SUMARIO'] == 'Número de Acórdão não utilizado')])
print(df_acordaos.shape[0], '\n')


imprime_proc_sigilo()

Qtde de acordãos sem número de processo:  380
Qtde de acordãos com tipo de processo SIGILOSO:  5201
Qtde de acordãos com acordao SIGILOSO:  1787
Qtde de acordãos com voto SIGILOSO:  2597
Qtde de acordãos com entidade SIGILOSO:  5542
Qtde de acordãos com Número de Acórdão não utilizado:  92 

Quantidade de acordãos marcados com sigilo:  5630 

Removendo os registros...
298562
293361
293087
292277
292207
292115 

Qtde de acordãos sem número de processo:  0
Qtde de acordãos com tipo de processo SIGILOSO:  0
Qtde de acordãos com acordao SIGILOSO:  0
Qtde de acordãos com voto SIGILOSO:  0
Qtde de acordãos com entidade SIGILOSO:  0
Qtde de acordãos com Número de Acórdão não utilizado:  0 

Quantidade de acordãos marcados com sigilo:  0 



In [11]:
pattern = re.compile('\D')


def format_numero_acordao(valor):
    string = re.sub(pattern, '', valor)
    if len(string)>4:
        return string[:-4]
    else:
        return string

In [12]:
df_acordaos.NUMERO_ACORDAO = df_acordaos.NUMERO_ACORDAO.apply(format_numero_acordao)

In [13]:
start_time = datetime.now()

df_acordaos['ID'] = df_acordaos.apply(tools.criaChaveAcordaoPrincipal, axis=1)

print('Duration: {}'.format(datetime.now() - start_time))

Duration: 0:00:09.701902


In [14]:
df_acordaos[df_acordaos.duplicated(['NUMERO_ACORDAO', 'PROCESSO', 'DATA_SESSAO'])]

,ID,URN,ANO_ACORDAO,NUMERO_ACORDAO,RELATOR,PROCESSO,TIPO_PROCESSO,DATA_SESSAO,NUMERO_ATA,ENTIDADE,ASSUNTO,SUMARIO,ACORDAO,RELATORIO,VOTO
294848,AC-002011-2019-PL,NA,2019,2011,AROLDO CEDRAZ,017.162/2007-1,Tomada de contas especial instaurada em razão ...,28/08/2019,33/2019-Plenário,Associação dos Irrigantes da Barragem de Terra...,Tomada de contas especial instaurada em razão ...,TOMADA DE CONTAS ESPECIAL. NÃO COMPROVAÇÃO DA ...,"VISTOS, relatados e discutidos estes autos de ...","Inicio este Relatório transcrevendo, com algun...","Conforme consignado no Relatório precedente, t..."
295941,AC-002011-2019-PL,NA,2019,2011,AROLDO CEDRAZ,017.162/2007-1,Tomada de contas especial instaurada em razão ...,28/08/2019,33/2019-Plenário,Associação dos Irrigantes da Barragem de Terra...,Tomada de contas especial instaurada em razão ...,TOMADA DE CONTAS ESPECIAL. NÃO COMPROVAÇÃO DA ...,"VISTOS, relatados e discutidos estes autos de ...","Inicio este Relatório transcrevendo, com algun...","Conforme consignado no Relatório precedente, t..."


In [15]:
df_acordaos.drop_duplicates(subset=['NUMERO_ACORDAO', 'PROCESSO', 'DATA_SESSAO'], keep='last', inplace=True)

In [16]:
print('Quantidade de ACÓRDÃOS nulos: ', df_acordaos[df_acordaos['ACORDAO'].isnull() == True].shape[0])
print('Quantidade de registros repetidos nos campos VOTO e ACÓRDAO:', df_acordaos[df_acordaos['VOTO'].eq(df_acordaos['ACORDAO'])].shape[0])
print('Quantidade de registros repetidos nos campos RELATÓRIO e ACÓRDAO:', df_acordaos[df_acordaos['RELATORIO'].eq(df_acordaos['ACORDAO'])].shape[0])

Quantidade de ACÓRDÃOS nulos:  23
Quantidade de registros repetidos nos campos VOTO e ACÓRDAO: 19
Quantidade de registros repetidos nos campos RELATÓRIO e ACÓRDAO: 18


In [17]:
df_acordaos['DECISAO'] = df_acordaos['ACORDAO'].map(str) + ' ' + df_acordaos['VOTO'].map(str) + ' ' + df_acordaos['RELATORIO'].map(str)

In [18]:
# Retira colunas que não interessam para o trabalho
df_acordaos = df_acordaos.drop(['URN', 'ANO_ACORDAO', 
                                'NUMERO_ACORDAO', 'RELATOR', 'PROCESSO', 'DATA_SESSAO', 'NUMERO_ATA', 
                                'ENTIDADE', 'SUMARIO', 'TIPO_PROCESSO', 'ACORDAO', 'VOTO', 'RELATORIO'], axis=1)

df_acordaos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 292113 entries, 0 to 298941
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   ID       292113 non-null  object
 1   ASSUNTO  260194 non-null  object
 2   DECISAO  292113 non-null  object
dtypes: object(3)
memory usage: 8.9+ MB


<h2>Trabalhando com o dataset de acórdãos rotulados

In [19]:
df_rotulados = pd.read_excel('dados/acordaos_rotulados.xlsx', sheet_name='Sheet1')
df_rotulados.insert(0, 'ID', '')

In [20]:
df_rotulados.rename(str.upper, axis='columns', inplace=True)
df_rotulados.rename(columns={'LABELCLASSE': 'CLASSE_TEMP', 'TÍTULO': 'TITULO', 'SUMÁRIO': 'SUMARIO', 
                            'TIPO DE PROCESSO': 'TIPO_PROCESSO',
                           'INTERESSADO / RESPONSÁVEL / RECORRENTE': 'INTERESSADO', 
                           'REPRESENTANTE DO MINISTÉRIO PÚBLICO': 'REP_MP',
                           'UNIDADE TÉCNICA': 'UNIDADE_TECNICA', 'REPRESENTANTE LEGAL': 'REP_LEGAL',
                           'ENDEREÇO DO ARQUIVO': 'FILE_PATH'}, inplace=True)


In [21]:
# Converte a classe em categorias
for i in range(0,14):
    df_rotulados.loc[df_rotulados['CLASSE_TEMP'] == 'classe' + str(i+1), ['CLASSE_TEMP']] = i+1
    
df_rotulados.CLASSE_TEMP.value_counts()

13    107
5      96
14     82
8      57
9      49
11     46
7      46
10     41
6      25
12     22
4      21
3      19
2      17
1       6
Name: CLASSE_TEMP, dtype: int64

In [22]:
# Insere o valor do campo ID
for index, row in df_rotulados.iterrows():
    df_rotulados.loc[index, 'ID'] = tools.criaChaveAcordao(df_rotulados.iloc[index].TITULO)

In [23]:
df_temp = df_rotulados.copy()

# Retira colunas que não interessam para o trabalho
df_rotulados = df_rotulados[['ID']]
df_rotulados.insert(1, 'CLASSE', '0')
df_rotulados.drop_duplicates(subset=['ID'], keep='first', inplace=True)
df_rotulados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 479 entries, 0 to 633
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      479 non-null    object
 1   CLASSE  479 non-null    object
dtypes: object(2)
memory usage: 11.2+ KB


In [24]:
def agrupar_multi_labels(df_base, df_compare):
    count = 0
    for index, row in df_base.iterrows():
        for index_comp, row_comp in df_compare.iterrows():
            try:
                if row.ID == row_comp.ID:
                    if row.CLASSE == '0':
                        df_base.loc[index]['CLASSE'] = str(row_comp.CLASSE_TEMP)
                    else:
                        df_base.loc[index]['CLASSE'] = df_base.loc[index]['CLASSE'] + ';' + str(row_comp.CLASSE_TEMP)
            except Exception as ex:
                print(ex)
    return df_base

In [25]:
start_time = datetime.now()
df_rotulados = agrupar_multi_labels(df_rotulados, df_temp)
print('Duration: {}'.format(datetime.now() - start_time))
del df_temp

Duration: 0:00:22.172912


In [26]:
df_left = pd.merge(df_acordaos, df_rotulados, on='ID', how='left')
del df_acordaos
df_left.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 292113 entries, 0 to 292112
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   ID       292113 non-null  object
 1   ASSUNTO  260194 non-null  object
 2   DECISAO  292113 non-null  object
 3   CLASSE   448 non-null     object
dtypes: object(4)
memory usage: 11.1+ MB


In [27]:
# Salva o arquivo em um csv
tools.save_file(df_left, 'dados/df_acordaos.csv')

Duration: 0:01:11.476355


In [28]:
new_columns = ['CLASSE1', 'CLASSE2', 'CLASSE3', 'CLASSE4']
df_temp = df_left[df_left.CLASSE.notnull()][['ID', 'CLASSE', 'ASSUNTO', 'DECISAO']]
df = pd.DataFrame(df_temp.CLASSE.str.split(';').tolist(), columns= new_columns)
df_temp = df_temp.assign(CLASSE1 = df.CLASSE1.tolist())
df_temp = df_temp.assign(CLASSE2 = df.CLASSE2.tolist())
df_temp = df_temp.assign(CLASSE3 = df.CLASSE3.tolist())
df_temp = df_temp.assign(CLASSE4 = df.CLASSE4.tolist())
del df_left
df_temp.drop('CLASSE', axis=1, inplace=True)

In [29]:
tools.save_file(df_temp, 'dados/df_rotulados.csv')
del df_temp

Duration: 0:00:00.972902
